In [1]:
import cerberus
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
import awsglue
import boto3
from cerberus import Validator
from awsglue.dynamicframe import DynamicFrame
from awsglue.context import GlueContext

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1611957411276_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# sc = SparkContext()
# glueContext = GlueContext(sc)
# spark = glueContext.spark_session
spark = SparkSession.builder.appName("Spark example").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# import boto3

# s3 = boto3.resource('s3')

# content_object = s3.Object('chen115y-test', 'nasdaq-sample-data/dq-test.txt')
# file_content = content_object.get()['Body'].read().decode('utf-8')
# # json_content = json.loads(file_content)
# print(file_content)

In [3]:
file_content = {
    'id': {
        'type': 'integer', 
        'min': 1, 
        'max': 10
    },
    'amount': {
        'type': 'float'
    },
    'first_order': {
        'type': 'datetime',
        'required': False
    },
    'user': {
        'type': 'string',
        'regex': '^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    },
    'items': {
        'type': 'list',
        'empty': False
    }
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
valid_entry = [{'id': 1, 'amount': 30.97, 'user': "ivan@amazon.com", 'items': ["1", "2"]}, \
               {'id': 20, 'amount': 30.97, 'user': "ivan#amazon.com", 'items': ["1", "2"]}]

df = spark.createDataFrame(valid_entry)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---+------+---------------+
|amount| id| items|           user|
+------+---+------+---------------+
| 30.97|  1|[1, 2]|ivan@amazon.com|
| 30.97| 20|[1, 2]|ivan#amazon.com|
+------+---+------+---------------+

In [ ]:
# One way to apply the data quality checking rules on the data frame
from cerberus import Validator
import json

dq_schema = eval(str(file_content))

results = []

V = Validator()
# schema_validator = Validator(schema)
for row in list(df.collect()):
    print(V.validate(row.asDict(), dq_schema))
    print(V.errors)
    results.append({'id': row['id'], 'error': json.dumps(V.errors)})

df_result = spark.createDataFrame(results)
df_result.toPandas().head()

In [11]:
# Another way (Spark way) to apply the data quality checking rules
# First, define a data quality checking function with input of wholse row
from cerberus import Validator
import json

def dq_check(allowed_per_unit):
    dq_schema = eval(str(file_content))
    V = Validator()
    is_valide = V.validate(row.asDict(), dq_schema)
    if not is_valide:
        result = V.errors
    else:
        result = ""
    return result

# Second, define the UDF function based on the data quality checking function. output type is a String.
dq_check_udf = udf(lambda x: dq_check(x) if x is not None else None, StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Apply the data quality checking rule and generate a new column for the checking result

df_result = df.withColumn("row_validation_error", dq_check_udf(col("allowed_per_unit")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Show the final data frame

df_result.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---+------+---------------+------------------------------------------------------------------------------------------------------------+
|amount|id |items |user           |row_validation_error                                                                                        |
+------+---+------+---------------+------------------------------------------------------------------------------------------------------------+
|30.97 |1  |[1, 2]|ivan@amazon.com|                                                                                                            |
|30.97 |20 |[1, 2]|ivan#amazon.com|{user=[value does not match regex '^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'], id=[max value is 10]}|
+------+---+------+---------------+------------------------------------------------------------------------------------------------------------+

In [17]:
df_result.write.csv("dq-results",mode="overwrite")